# Introducción al Machine Learning

La idea es usar tecnicas de **Machine Learning** para determinar qué propiedades fisicoquímicas hacen a la calidad de un vino.

Esta tarea fue publicada originalmente en: *P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.*

In [ ]:
# Importa las librerias a utilzar
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.2)
from sklearn.metrics import accuracy_score ,confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Carga el dataset de vinos rojos 
data_red = pd.read_csv("./data/winequality-red.csv", sep=';')
# Muestra los primeros 5 registros
data_red.head(n=5)

In [ ]:
data_red.shape

# Parte 1: Exploracion de los datos

In [ ]:
# accede a la Serie de "quality"
data_red["quality"]

In [ ]:
# cuenta cuantos vinos hay de cada calidad
data_red["quality"].value_counts()

In [ ]:
# grafico de barras
data_red["quality"].value_counts().sort_index().plot.bar(color="darkgreen");

In [ ]:
# agrega una nueva columna llamado "calidad".
# Baja: quality < 5.5
# Alta: quality > 5.5
data_red["calidad"] = pd.cut(data_red['quality'], bins=[0, 5.5, 10], labels=["Baja","Alta"])

In [ ]:
data_red.head()

### Separacion en Training y Testing sets: 

In [ ]:
# separa en features (X) y etiquetas (Y)
X = data_red.iloc[:,:11]
Y = data_red["calidad"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
data_train = pd.concat([X_train,y_train],axis=1)
data_train.head()

### Exploración de los features

In [ ]:
# Por cada feature se hace n dos histograma

# configura los los paneles
plt.subplots(ncols=4,nrows=3,figsize=(16,16))
# itera por todos los features
for i,columna in enumerate(data_train.columns[:11]):
    #selecciona el panel
    plt.subplot(4,3,i+1)
    
    # grafica los dos histogramas
    _, bins, _ = plt.hist(data_train[data_train.calidad=="Alta"][columna],label="Alta", alpha=0.5, density=True)
    plt.hist(data_train[data_train.calidad=="Baja"][columna], bins=bins, label="Baja", alpha=0.5, density=True);
    # incluye titulo
    plt.title(columna)
    # incluye leyenda
    plt.legend()

# ajusta los paneles
plt.tight_layout()

### Scatterplots y Densidades

In [ ]:
# Scatterplot
ax = sns.scatterplot(x="sulphates", y="volatile acidity", data=data_train[data_train.calidad=="Alta"],color="blue",alpha=0.5,s=10,label="Alta")
sns.scatterplot(x="sulphates", y="volatile acidity", data=data_train[data_train.calidad=="Baja"],ax=ax,color="red",alpha=0.5,s=10,label="Baja")

# setea limites de los ejes
ax.set_xlim((0.25,1));
ax.set_ylim((0.1,1.25));

# ajusta bordes
plt.tight_layout()

# guarda figura
plt.savefig("red_wine_scatterplot1.png")

In [ ]:
# Separa el dataset segun la calidad
data_alta = data_train[data_train.calidad=="Alta"]
data_baja = data_train[data_train.calidad=="Baja"]

# graficos de densidades
sns.kdeplot(data_alta["sulphates"], data_alta["volatile acidity"],cmap="Blues")
ax = sns.kdeplot(data_baja["sulphates"], data_baja["volatile acidity"],cmap="Reds")

# setea limites de los ejes
ax.set_xlim((0.25,1));
ax.set_ylim((0.1,1.25));

In [ ]:
# Separa el dataset segun la calidad
data_alta = data_train[data_train.calidad=="Alta"]
data_baja = data_train[data_train.calidad=="Baja"]

# scatterplots
ax = sns.scatterplot(x="sulphates", y="volatile acidity", data=data_alta,color="blue",alpha=0.5,s=10,label="Alta")
sns.scatterplot(x="sulphates", y="volatile acidity", data=data_baja,ax=ax,color="red",alpha=0.5,s=10,label="Baja")

# graficos de densidades
sns.kdeplot(data_alta["sulphates"], data_alta["volatile acidity"],cmap="Blues")
sns.kdeplot(data_baja["sulphates"], data_baja["volatile acidity"],cmap="Reds")

# setea limites de los ejes
ax.set_xlim((0.25,1));
ax.set_ylim((0.1,1.25));

### No todos los features son tan utiles para separar las clases

In [ ]:
# scatterplots
ax = sns.scatterplot(x="free sulfur dioxide", y="pH", data=data_alta,color="blue",alpha=0.5,s=10,label="Alta");
sns.scatterplot(x="free sulfur dioxide", y="pH", data=data_baja,ax=ax,color="red",alpha=0.5,s=10,label="Baja");

# graficos de densidades
sns.kdeplot(data_alta["free sulfur dioxide"], data_alta["pH"],cmap="Blues")
sns.kdeplot(data_baja["free sulfur dioxide"], data_baja["pH"],cmap="Reds")

# setea limites de los ejes
ax.set_xlim((1.2,30));
ax.set_ylim((2.9,3.8));

# ajusta bordes
plt.tight_layout()

# guarda figura
plt.savefig("red_wine_scatterplot2.png")

## Clasificación de vinos con lineas verticales y horizontales
### Ejemplo con dos features

In [ ]:
# graficos de densidades
sns.kdeplot(data_alta["sulphates"], data_alta["alcohol"],cmap="Blues")
ax = sns.kdeplot(data_baja["sulphates"], data_baja["alcohol"],cmap="Reds")

# scatterplots
sns.scatterplot(x="sulphates", y="alcohol", data=data_alta,ax=ax,color="blue",alpha=0.7,s=10,label="Alta");
sns.scatterplot(x="sulphates", y="alcohol", data=data_baja,ax=ax,color="red",alpha=0.7,s=10,label="Baja");

# setea limites de los ejes
ax.set_xlim((0.25,1.5));
ax.set_ylim((8,15))

In [ ]:
# graficos de densidades
sns.kdeplot(data_alta["sulphates"], data_alta["alcohol"],cmap="Blues")
ax = sns.kdeplot(data_baja["sulphates"], data_baja["alcohol"],cmap="Reds")

# scatterplots
sns.scatterplot(x="sulphates", y="alcohol", data=data_alta,ax=ax,color="blue",alpha=0.7,s=10,label="Alta");
sns.scatterplot(x="sulphates", y="alcohol", data=data_baja,ax=ax,color="red",alpha=0.7,s=10,label="Baja");

# setea limites de los ejes
ax.set_xlim((0.25,1.5));
ax.set_ylim((8,15))

# agrega linea vertical y horizontal
plt.vlines(x=0.7,ymin=8, ymax=15,color="black")
plt.axhline(y = 10,color="black")


In [ ]:
# arma un clasificador con reglas
def detector_de_vinos_de_calidad_alta(x):
    if x["alcohol"]>10:
        return "Alta"
    elif x["sulphates"]>0.7:
        return "Alta"
    else:
        return "Baja"

In [ ]:
# clasifica los vinos del test set con mi "detector_de_vinos_de_calidad_alta"
y_predicted = X_test.apply(detector_de_vinos_de_calidad_alta,axis=1)
# calcula precision, recall y f-score
print("precision:",precision_score(y_test,y_predicted,pos_label="Alta") )
print("recall:",recall_score(y_test,y_predicted,pos_label="Alta") )
print("f-score:",f1_score(y_test,y_predicted,pos_label="Alta") ) 


In [ ]:
# Arma la matriz de confusion
CM = confusion_matrix(y_test,y_predicted, labels=["Alta","Baja"])
pd.DataFrame(CM,index=["True Alta","True Baja"],columns=["predicted Alta","predicted Baja"])

#### Comparo con una clasificacion Random

In [ ]:
# clasificador aleatoreo que mantiene la proporcion de vinos buenos
calidad_random = np.random.permutation(np.array(y_test))

# calcula el: precision, recall y f-score
print("precision:",precision_score(y_test,calidad_random,pos_label="Alta") )
print("recall:",recall_score(y_test,calidad_random,pos_label="Alta") )
print("f-score:",f1_score(y_test,calidad_random,pos_label="Alta") ) 

In [ ]:
# Matriz de confusion con las clasificaciones shufleadas
CM_random = confusion_matrix(y_test,calidad_random, labels=["Alta","Baja"])
pd.DataFrame(CM_random,index=["True Alta","True Baja"],columns=["predicted Alta","predicted Baja"])

# Hacer esto con todas las variables es demasiado tedioso e impreciso
# El Machine Learning viene a hacer este trabajo

<img src="/data/images.png">

---------------------------------------

# Parte 2: Machine Learning

In [ ]:
# pasa a booleanos las etiquetas (True si la calidad es alta, sino False)
y_train_bool = y_train=="Alta" 
y_test_bool = y_test=="Alta" 

## Arboles de Decision

In [ ]:
# define el modelo de Clasificacion
clf_tree = DecisionTreeClassifier()

# parametros a variar 
parameters = {'max_features': [3,5,7],"max_depth":[5,10,None] }

# define un grid search a realizar 
grid_search_tree = GridSearchCV(clf_tree, parameters,scoring='f1',cv=5,n_jobs=-1)

# realiza el grid search evaluando el modelo en un esquema de Cross-Validation
grid_search_tree.fit(X_train, y_train_bool)

# selecciona el mejor modelo (mejor set de parametros)
best_clf_tree = grid_search_tree.best_estimator_
print("max_features:",best_clf_tree.max_features,"; max_depth:",best_clf_tree.max_depth)

#### Ahora puedo "predecir" si un nuevo vino es de alta calidad

In [ ]:
X_test.iloc[0]

In [ ]:
# clasifica el primer elemento del testing set
best_clf_tree.predict([X_test.iloc[0]])

In [ ]:
# clasifica el test-set usando el mejor modelo
predictions_tree = best_clf_tree.predict(X_test)
print("f-score =",f1_score(y_test_bool, predictions_tree))

In [ ]:
# Calcula el fscore y la Matriz de Confusion
CM_Tree= confusion_matrix(y_test_bool,predictions_tree)
pd.DataFrame(CM_Tree,index=["True Alta","True Baja"],columns=["predicted Alta","predicted Baja"])

In [ ]:
# Visualiza el arbol de decision
import graphviz 
from sklearn import tree
arbol = tree.export_graphviz(best_clf_tree, out_file=None,feature_names=X_train.columns,filled=True, rounded=True)  
graphviz.Source(arbol).render("arbol_de_decision") 

## Modelos mas Interesantes (Random Forest)

In [ ]:
# define el modelo de Clasificacion
clf_RF= RandomForestClassifier(class_weight="balanced")
# parametros a variar 
parameters = {'n_estimators':[100],'max_features': [3,5,7],"max_depth":[5,10,None] }
# define un grid search a realizar 
grid_search_RF = GridSearchCV(clf_RF, parameters,scoring='f1',cv=5,n_jobs=-1)
# realiza el grid search evaluando el modelo en un esquema de Cross-Validation
grid_search_RF.fit(X_train, y_train_bool)
# selecciona el mejor modelo (mejor set de parametros)
best_clf_RF = grid_search_RF.best_estimator_
print("n_estimators:",best_clf_RF.n_estimators,"; max_features:",best_clf_RF.max_features,"; max_depth:",best_clf_RF.max_depth)
# Clasifica el test-set usando el mejor modelo
predictions_RF = best_clf_RF.predict(X_test)


In [ ]:
# Calcula el fscore y la Matriz de Confusion
print ("f-score =",f1_score(y_test_bool, predictions_RF))
CM_RF = confusion_matrix(y_test_bool,predictions_RF, labels=[True,False])
pd.DataFrame(CM_RF,index=["True Alta","True Baja"],columns=["predicted Alta","predicted Baja"])

## Calculo de la importancia de los feature


In [ ]:
pd.Series(best_clf_RF.feature_importances_,index=X_train.columns).sort_values(ascending=False).plot(kind="bar",color="darkgreen");

## Comparacion de Modelos

In [ ]:
resultados = pd.Series({"Random":f1_score(y_test_bool,calidad_random=="Alta"),
           "A_ojo":f1_score(y_test_bool,y_predicted=="Alta"),
           "Arboles":f1_score(y_test_bool,predictions_tree),
           "Random Forest":f1_score(y_test_bool,predictions_RF)})
resultados.plot(kind="bar",color="Darkgreen")
plt.ylabel("F-score"); plt.ylim((0.5));